## 环境设置
%conda create -n distilabel python=3.12.9
%conda activate distilabel

## 安装基础sdk包

In [ ]:
%pip install distilabel[ollama,openai]

## 先测试一下普通llm的应答效果

In [32]:
from distilabel.models.llms import OllamaLLM
from distilabel.models.llms import OpenAILLM

llm = OllamaLLM(model="llama3.2-vision:latest")
#llm = OllamaLLM(model="deepseek-r1:7b")
#llm = OpenAILLM(model="deepseek-r1:7b",base_url="http://127.0.0.1:11434/v1",api_key="akey")
llm.load()

# Call the model
output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": "Hello world!"}]])
print(output[0]['generations'][0])

C:\Users\13802\AppData\Local\Temp\ipykernel_23860\786400136.py:4: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  llm = OllamaLLM(model="llama3.2-vision:latest")


你好，世界！


## deepseek 的响应
7b的响应不正常，14b的响应相对ok

In [ ]:
from distilabel.models.llms import OllamaLLM
from distilabel.models.llms import OpenAILLM

#llm = OllamaLLM(model="llama3.2-vision:latest")
llm = OllamaLLM(model="deepseek-r1:14b")
#llm = OpenAILLM(model="deepseek-r1:7b",base_url="http://127.0.0.1:11434/v1",api_key="akey")
llm.load()
# Call the model
output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": "Hello world!"}]])
print(output)

extracted_content = output[0]['generations'][0].split('</think>\n\n')[1]
print(extracted_content)

## load dataset

In [ ]:
from distilabel.steps import LoadDataFromHub
import os
os.environ["ALL_PROXY"] = "http://127.0.0.1:1087"
loader = LoadDataFromHub(
    repo_id="Conard/fortune-telling",
    split="train")
loader.load()
(result,_)=next(loader.process())
for item in result:
    print(item['Question'])
    print(item['Response'])
    print(item['Complex_CoT'])


对比大模型和语料输出

In [ ]:
from distilabel.steps import LoadDataFromHub
import os
from distilabel.models.llms import OllamaLLM
from distilabel.models.llms import OpenAILLM
import warnings 
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 





os.environ["ALL_PROXY"] = "http://127.0.0.1:1087"
loader = LoadDataFromHub(
    repo_id="Conard/fortune-telling",
    split="train")
loader.load()
os.environ['ALL_PROXY'] = ''
(result,_)=next(loader.process())
for item in result:
    print('----------------------------------------------------------')
    print(f'问题:{item["Question"]}')
    print('----------------------------------------------------------')
    print(f'数据集的应答:{item["Response"]}')
    print('----------------------------------------------------------')
    llm = OllamaLLM(model="deepseek-r1:14b")
    llm.load()
    output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": item["Question"]}]])
    cot = output[0]['generations'][0].split('</think>\n\n')[0]
    extracted_content = output[0]['generations'][0].split('</think>\n\n')[1]
    print('----------------------------------------------------------')

    print(f'ds大模型的应答:{extracted_content}')
    output = llm.generate(inputs=[[{"role": "system", "content": "You are a helpful assistant,you always response in chinese"},{"role": "user", "content": item["Question"]+cot}]])
    #extracted_content = output[0]['generations'][0].split('</think>\n\n')[1]
    print('----------------------------------------------------------')
    print(f'ds大模型使用cot的应答:{output[0]['generations'][0]}')
    #print(f'数据集的思维链:{item["Complex_CoT"]}')
    break

Step 'None' hasn't received a pipeline, and it hasn't been created within a `Pipeline` context. Please, use `with Pipeline() as pipeline:` and create the step within the context.


----------------------------------------------------------
问题:新房装修,大门对着电梯好不好?要如何化解?
----------------------------------------------------------
数据集的应答:根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：

1. 玄关阻隔法
在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带

2. 五行通关法
门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜

3. 光影化解术
安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域

4. 现代科技方案
入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场

需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。
----------------------------------------------------------


Exception ignored in: <function AsyncLLM.__del__ at 0x0000015900BFB6A0>
Traceback (most recent call last):
  File "c:\Users\13802\miniconda3\envs\distilabel\Lib\site-packages\distilabel\models\llms\base.py", line 421, in __del__
    if self._new_event_loop:
       ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\13802\miniconda3\envs\distilabel\Lib\site-packages\pydantic\main.py", line 870, in __getattr__
    return self.__pydantic_private__[item]  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\13802\miniconda3\envs\distilabel\Lib\site-packages\pydantic\main.py", line 888, in __getattr__
    return super().__getattribute__(item)  # Raises AttributeError if appropriate
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'OllamaLLM' object has no attribute '__pydantic_private__'
